In [27]:
import pandas as pd

In [28]:
data = pd.read_csv("BDD/avis_sans_outliers.csv")

In [29]:
# Drop unnecessary columns and handle missing values
data_cleaned = data.drop(columns=['Unnamed: 0', 'url', 'title_review', 'date_published'])
data_cleaned['comment'] = data_cleaned['comment'].fillna('')  # Fill missing comments

In [34]:
from surprise import Dataset, Reader, KNNWithZScore
from collections import defaultdict

# Prepare data for surprise
reader = Reader(rating_scale=(1, 10))  # Assuming rating scale is from 1 to 10
data_surprise = Dataset.load_from_df(data[['author', 'title', 'note']], reader)

# Build full trainset
trainset = data_surprise.build_full_trainset()

# Initialize KNN with Z-Score algorithm for user-based collaborative filtering
algo = KNNWithZScore(sim_options={'name': 'cosine', 'user_based': True}, k=20, min_k=1)
algo.fit(trainset)

def get_neighbors(user_id, game_title, k=20):
    # Retrieve inner ID of the user
    user_inner_id = trainset.to_inner_uid(user_id)
    
    # Retrieve inner ID of the game
    try:
        game_inner_id = trainset.to_inner_iid(game_title)
    except ValueError:
        return f"No data available for the game '{game_title}'."
    
    # Retrieve the k nearest neighbors of the user
    neighbors = algo.get_neighbors(user_inner_id, k)
    # Convert inner IDs of the neighbors back to raw IDs
    neighbors_ids = [trainset.to_raw_uid(inner_id) for inner_id in neighbors]
    print(neighbors_ids)
    # Filter the dataset to find the neighbors who have rated the specified game
    filtered_comments = data[(data['author'].isin(neighbors_ids)) & (data['title'] == game_title)]
    
    # Collect and return usernames and their comments on the specified game
    return filtered_comments[['author', 'comment']].values.tolist()


Computing the cosine similarity matrix...
Done computing similarity matrix.


In [62]:
neighbors_comments = get_neighbors('Monsieur Guillaume', 'Mariposas', k=20)
print(neighbors_comments)

[4, 11, 14, 15, 16, 18, 19, 21, 24, 28, 30, 36, 44, 46, 47, 48, 49, 50, 51, 58]
['prunelles', 'Olfenw', 'Lilly', 'letroyenfou', 'Empédocle', 'Ricardo31', 'Player One', 'Sempre Sainté', 'grotesk', 'Olène', 'Greta', 'Patmol', 'Delorean', 'plumesdanges', 'Didi5962', 'Sirien', 'Pollo', 'Meeplejuice', 'Gido_L4', 'Tigragon22']
[['prunelles', 'Très joli bijou que ce jeu-là ! Le matériel, les illustrations, l\'histoire et le propos du jeu, tout donne envie de contribuer à sauver ces papillons.\\nBelle courbe d\'apprentissage, impossible de tout faire même si on essaie parfois.\\nLe mini bemol serait les papillons qu\'on n\'arrive pas toujours à visualiser parce que certaines couleurs sont proches mais en même ça nous permet de les camoufler aussi aux yeux de nos adversaires.\\nPensez à aller regarder les différentes variantes concernant les cartes "cycle de vie". \\nUne belle réussite !'], ['Olfenw', nan], ['Lilly', "Mariposas est un jeu ou la tension va s'exacerber au cours de la partie, ou d

In [55]:
data[['title','comment']].loc[data['author']=='Monsieur Guillaume']

,title,comment
0,Mariposas,"Lorsque le jeu est jeu, bon, réflexif, joli po..."
73,The LOOP,"Edition (Univers, Direction artistique, Matéri..."
205,Harry Potter : Bataille à Poudlard - Défense c...,"Edition (Univers, Direction artistique, Matéri..."
216,Roméo & Juliette,Marions (c'est le cas de le dire) un thème fin...
308,Tekhenu,"Oui, parce que là, il faudra un peu vous masse..."
1046,Clank! Legacy: Acquisitions Incorporated.,C'est avec Renegade France que nous repartons ...
1426,Schotten Totten 2,"Edition (Univers, Direction artistique, Matéri..."
1587,Prehistories,"Edition (Univers, Direction artistique, Matéri..."
1649,The Crew,"Très bien vu, Monsieur Sing ! Depuis votre con..."
1825,MicroMacro Crime City,"Edition (Univers, Direction artistique, Matéri..."
